# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_lf.ipynb
- template script for analyzing direct arrival waveforms from Sumatra:
- channel LHZ, duration up to 10.5 days

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt 
import numpy as np
import warnings

from obspy.core import read
from obspy.core import UTCDateTime

from pysep import Pysep
from pysep.recsec import plotw_rs
from plot_event_station import plot_event_station

from lib_seis import get_dist_az
from lib_seis import plot_event_station
from lib_seis import station_info_list
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft

In [ ]:
warnings.filterwarnings('ignore')                  
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# setting event and data parameters

origin_time        = UTCDateTime('2004,12,26,00,58,53')
event_latitude     = 3.09
event_longitude    = 94.26
event_depth_km     = 30
event_magnitude    = 9.1
resample_freq      = 1
data_dir           = 'datawf/sumatra_LHZ_vel/SAC'

In [ ]:
# download LHZ data, with response removed to your chosen output_unit
# choose your own inputs below

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime('2004,12,26,00,58,53'),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = '*',
                  locations                    = '*',
                  channels                     = 'LHZ',
                  mindistance                  = 0,
                  maxdistance                  = 3000,
                  remove_response              = True,
                  output_unit                  = 'VEL',
                  remove_clipped               = True,
                  remove_insufficient_length   = True,
                  detrend                      = True,
                  demean                       = True,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 0.03,
                  resample_freq                = 1,
                  seconds_before_ref           =  0.5 * 24 * 60 * 60,
                  seconds_after_ref            = 10.0 * 24 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'DEBUG',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite_event_tag          = 'sumatra_LHZ_vel')

try:
    ps = Pysep(**download)
    ps.run()
except :
    print('data directory already exists')

In [ ]:
# read downloaded LHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
print('\nSource receiver map')
event_latitude  = download['event_latitude']
event_longitude = download['event_longitude']
station_lats, station_lons, station_tags, station_tags_full = station_info_list(st)
plot_event_station(event_latitude,event_longitude,slat=station_lats,slon=station_lons,stas=station_tags)

In [ ]:
print('Table of station azimuths and distances\n')
distance_deg, azimuth_deg, distance_km = get_dist_az(event_latitude,event_longitude,station_lats,station_lons,station_tags_full)

In [ ]:
# plot the record section using Pyseps's record section plotting tool
# https://adjtomo.github.io/pysep/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection

plot_info = dict( pysep_path                   = data_dir,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  min_period_s                 = 100,
                  max_period_s                 = 1000,
                  preprocess                   = 'st',
                  max_traces_per_rs            = None,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'CRITICAL')

plotw_rs(**plot_info)